In [3]:
import numpy as np
import os

In [4]:
%pwd

'c:\\Users\\Amit\\Notebooks\\Alianz_Assignment\\E2E_Term_Scheme\\research'

In [5]:
os.chdir("../")

In [6]:
# ENTITY - creating datatype of configurations
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)  # no need of self, useful in defining data_type
class DataingestionConfig:
  root_dir: Path
  source_url: str
  local_data_file: Path
  unzip_dir : Path 

In [7]:
from mlproject.constants import *
from mlproject.utils.common import read_yaml,create_directories

#print(CONFIG_FILE_PATH)

In [8]:
# SRC/CONFIG -> to take the configurations and take actions according to the configs 
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH ):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataingestionConfig:

        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataingestionConfig(
            root_dir = config.root_dir,
            source_url = config.source_url,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir,
        )

        return data_ingestion_config
    

In [9]:
import os
import urllib.request as request
import zipfile
from mlproject import logger
from mlproject.utils.common import get_size

In [10]:
class Dataingestion:
    def __init__(self, config:DataingestionConfig):
        self.config = config

    def download_file(self):

        if not os.path.exists(self.config.local_data_file):
            filename , headers = request.urlretrieve(
                url= self.config.source_url,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download ! with the info:\n {headers}")
        else:
            logger.info(f"file already exists of size {get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = Dataingestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
        raise e

[2024-05-20 17:39:07,681: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-20 17:39:07,681: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-20 17:39:07,686: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-05-20 17:39:07,688: INFO: common: created directory at: artifacts]
[2024-05-20 17:39:07,688: INFO: common: created directory at: artifacts/data_ingestion]
[2024-05-20 17:39:08,693: INFO: 236588655: artifacts/data_ingestion/data.zip download ! with the info:
 Connection: close
Content-Length: 357766
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "208fa42863452f47295cab4288ba323c6bdad4c9b561bdce2eed0187f47f134b"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: E6BA:14FBEE:19D74F:2377FD:664B3D64
Accept-Ranges: bytes
Date: Mon, 20 May 